In [88]:
from langchain_huggingface import HuggingFaceEndpoint, ChatHuggingFace
from langchain_core.tools import tool
from langchain_core.messages import HumanMessage
import requests
from langchain_openai import ChatOpenAI

In [ ]:
api_key = ''

In [83]:
from langchain_core.tools import InjectedToolArg
from typing import Annotated

@tool
def get_conversion_factor(base_currency: str, target_currency:str)->float:
    '''this fn fetches the currency conversion factor b/w a given base currency and a target currency'''
    url = f'https://v6.exchangerate-api.com/v6/2c5f61cd8017b794f948ae40/pair/{base_currency}/{target_currency}'
    response = requests.get(url)
    return response.json()

@tool
def convert(base_currency:int,conversion_rate:Annotated[float,InjectedToolArg])->float: #llm don ono try to fill this argument i am the developer will inject this value after ruinning earlier tools
    '''given a currency conversion rate this fn calculate the target currency value from the given base_currency'''
    return base_currency*conversion_rate

In [84]:
get_conversion_factor.invoke({'base_currency':'USD','target_currency':'INR'})

{'result': 'success',
 'documentation': 'https://www.exchangerate-api.com/docs',
 'terms_of_use': 'https://www.exchangerate-api.com/terms',
 'time_last_update_unix': 1748044801,
 'time_last_update_utc': 'Sat, 24 May 2025 00:00:01 +0000',
 'time_next_update_unix': 1748131201,
 'time_next_update_utc': 'Sun, 25 May 2025 00:00:01 +0000',
 'base_code': 'USD',
 'target_code': 'INR',
 'conversion_rate': 85.4133}

In [ ]:
model = ChatOpenAI(
    base_url="https://openrouter.ai/api/v1",
    model="openai/gpt-3.5-turbo",
    api_key=api_key
)

In [94]:
model_with_tool = model.bind_tools([get_conversion_factor,convert])

In [95]:
messages = [HumanMessage('what is the conversion factor between USD and INR, and based on that can You convert 10 usd to inr')]

In [96]:
messages

[HumanMessage(content='what is the conversion factor between USD and INR, and based on that can You convert 10 usd to inr', additional_kwargs={}, response_metadata={})]

In [97]:
ai_message = model_with_tool.invoke(messages)

In [110]:
messages.append(ai_message)

In [98]:
ai_message.tool_calls

[{'name': 'get_conversion_factor',
  'args': {'base_currency': 'USD', 'target_currency': 'INR'},
  'id': 'call_jREiZ9UeLVQdZDzx29kuoiFa',
  'type': 'tool_call'},
 {'name': 'convert',
  'args': {'base_currency': 10},
  'id': 'call_feVePfvRJ8g4KZ2hXGGYyz2R',
  'type': 'tool_call'}]

In [ ]:
import json
for tool_call in ai_message.tool_calls:
    # execute the 1st tool and get the value of conv rate
    if tool_call['name'] == 'get_conversion_factor':
        tool_msg1 = get_conversion_factor.invoke(tool_call)
        # fetch conversion rate
        parser = json.loads(tool_msg1.content)
        conversion_rate = parser['conversion_rate']
        # append msg to msg list
        messages.append(tool_msg1)
    # execute the 2nd with conversion rate from tool 1
    if tool_call['name'] == 'convert':
        # fetch the current args
        tool_call['args']['conversion_rate'] = conversion_rate
        tool_msg2 = convert.invoke(tool_call)
        messages.append(tool_msg2)

content='{"result": "success", "documentation": "https://www.exchangerate-api.com/docs", "terms_of_use": "https://www.exchangerate-api.com/terms", "time_last_update_unix": 1748044801, "time_last_update_utc": "Sat, 24 May 2025 00:00:01 +0000", "time_next_update_unix": 1748131201, "time_next_update_utc": "Sun, 25 May 2025 00:00:01 +0000", "base_code": "USD", "target_code": "INR", "conversion_rate": 85.4133}' name='get_conversion_factor' tool_call_id='call_jREiZ9UeLVQdZDzx29kuoiFa'


In [114]:
messages

[HumanMessage(content='what is the conversion factor between USD and INR, and based on that can You convert 10 usd to inr', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_jREiZ9UeLVQdZDzx29kuoiFa', 'function': {'arguments': '{"base_currency": "USD", "target_currency": "INR"}', 'name': 'get_conversion_factor'}, 'type': 'function', 'index': 0}, {'id': 'call_feVePfvRJ8g4KZ2hXGGYyz2R', 'function': {'arguments': '{"base_currency": 10}', 'name': 'convert'}, 'type': 'function', 'index': 1}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 51, 'prompt_tokens': 122, 'total_tokens': 173, 'completion_tokens_details': {'accepted_prediction_tokens': None, 'audio_tokens': None, 'reasoning_tokens': 0, 'rejected_prediction_tokens': None}, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}}, 'model_name': 'openai/gpt-3.5-turbo', 'system_fingerprint': None, 'id': 'gen-1748075044-rZjCqBXdF3fy07r

In [115]:
result = model_with_tool.invoke(messages)

In [116]:
result.content

'The conversion factor between USD and INR is 85.4133. \n\nBased on that, 10 USD is equivalent to 854.133 INR.'